In [62]:
import pandas as pd
import numpy as np
from Levenshtein import distance as levenshtein_distance
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
raw_dataset = pd.read_csv('data/pf_suvs_i302_1s2025.csv')

# Limpieza del dataset

Reemplazamos la muestra con 'Marca' = 'Vol' por 'Volkswagen'.

In [65]:
raw_dataset["Marca"] = raw_dataset["Marca"].replace("Vol", "Volkswagen")

Extraemos la cantidad de caracteres de la feature 'Descripción'

In [66]:
raw_dataset['Caracteres_descripcion'] = raw_dataset['Descripción'].apply(lambda x: len(str(x)))

Pasar el precio de pesos a dólares.

In [67]:
precio_dolar = 1150
mask = raw_dataset['Moneda'] == '$'
raw_dataset.loc[mask, 'Precio'] = raw_dataset.loc[mask, 'Precio'] / precio_dolar

Procesamiento del color

In [68]:
for i in range(len(raw_dataset)):
    if raw_dataset['Color'][i] == np.nan:
        raw_dataset['Color'][i] = 'Otro'

known_colors = ['Blanco', 'Negro', 'Gris', 'Plata', 'Acero', 'Azul', 'Gray', 'Rojo', 
                'Granito', 'Beige', 'Verde', 'Marrón', 'Amarillo', 'Violeta', 'Naranja', 
                'Dorado', 'Grafito', 'White', 'Cobre', 'Black', 'Rosa', 'Celeste', 'Morado', 
                'Plateado', 'Celeste', 'Beige', 'blue', 'Bordó', 'Celeste', 'Otro']

for color in known_colors:
    raw_dataset[color] = 0

for i, color_input in enumerate(raw_dataset['Color']):
    dists = [levenshtein_distance(str(color_input).lower(), color.lower()) for color in known_colors]
    idx_min = np.argmin(dists)
    matched_color = known_colors[idx_min]
    raw_dataset.at[i, matched_color] = 1

Limpiar la columna de 'Kilómetros'

In [69]:
def clean_km_value(value):
    """Limpia un valor individual de kilómetros"""
    if pd.isna(value):
        return np.nan
    
    value_str = str(value).strip()
    
    if value_str.endswith(' km'):
        value_str = value_str[:-3].strip()
    if value_str.endswith('km'):
        value_str = value_str[:-2].strip()
    
    if '.' in value_str:
        parts = value_str.split('.')
        if len(parts) == 2:
            if parts[1] == '0':
                return float(parts[0])
            elif len(parts[1]) == 3 and parts[1].isdigit():
                return float(parts[0] + parts[1])
            else:
                return float(value_str)
    
    try:
        return float(value_str)
    except ValueError:
        return np.nan

def clean_kilometers(raw_dataset):
    """
    Limpia la columna 'Kilómetros' removiendo 'km' y '.0', 
    y convirtiendo correctamente valores como '37.000 km' a 37000
    """
    df = raw_dataset.copy()
    
    df['Kilómetros'] = df['Kilómetros'].apply(clean_km_value)
    
    return df

np.set_printoptions(suppress=True, formatter={'float': '{:.0f}'.format})

raw_dataset = clean_kilometers(raw_dataset)

Ver la 'Marca' con la distancia de levenshtein a los nombres de las marcas más conocidas

In [70]:
known_brands = [
  'Ford', 'Volkswagen', 'Jeep', 'BAIC', 'Kia', 'Hyundai', 'Porsche',
  'Peugeot', 'Fiat', 'Chevrolet', 'Citroën', 'BMW', 'Audi', 'Honda',
  'Nissan', 'Mercedes-Benz', 'Renault', 'Suzuki', 'Toyota',
  'Chery', 'Daihatsu', 'SsangYong', 'Dodge', 'JAC', 'Land Rover',
  'Alfa Romeo', 'Haval', 'Volvo', 'Lifan', 'Mini', 'D·S',
  'Mitsubishi', 'Jetour', 'GWM', 'KAIYI',
  'Lexus', 'Isuzu', 'Subaru', 'Jaguar'
]

In [71]:
for brand in known_brands:
    raw_dataset[brand] = 0

for i, marca_input in enumerate(raw_dataset['Marca']):
    dists = [levenshtein_distance(str(marca_input).lower(), brand.lower()) for brand in known_brands]
    idx_min = np.argmin(dists)
    matched_brand = known_brands[idx_min]
    raw_dataset.at[i, matched_brand] = 1

Notamos que el 'Tipo de carrocería' es para todas las observaciones igual a SUV, entonces tiramos la columna.

In [72]:
raw_dataset.drop('Tipo de carrocería', axis=1, inplace=True)

Con el feature de 'Modelo' hacemos lo mismo que para la 'Marca'.

In [73]:
known_models = [
    '2008', '208', '3008', '4008', '4Runner', '500X', 'Actyon', 'Blazer', 'Bronco',
    'Bronco Sport', 'C3', 'C3 Aircross', 'C4 Aircross', 'C4 Cactus', 'C5 Aircross',
    'CR-V', 'Captur', 'Cayenne', 'Cherokee', 'Clase GL', 'Clase GLA', 'Clase GLC',
    'Clase GLE', 'Clase GLK', 'Commander', 'Compass', 'Cooper Countryman', 'Corolla Cross',
    'Creta', 'DS3', 'DS7', 'DS7 Crossback', 'Defender', 'Discovery', 'Duster',
    'Duster Oroch', 'Ecosport', 'Equinox', 'Evoque', 'Explorer', 'F-PACE', 'Freelander',
    'Galloper', 'Grand Blazer', 'Grand Cherokee', 'Grand Santa Fé', 'Grand Vitara',
    'H6', 'HR-V', 'Hilux SW4', 'Jimny', 'Jolion', 'Journey', 'Kicks', 'Koleos',
    'Kona', 'Kuga', 'Land Cruiser', 'ML', 'Macan', 'Mohave', 'Montero', 'Murano',
    'Musso', 'Myway', 'NX', 'Nativa', 'Nivus', 'Outback', 'Outlander', 'Panamera',
    'Pathfinder', 'Patriot', 'Pilot', 'Pulse', 'Q2', 'Q3', 'Q3 Sportback', 'Q5', 'Q7',
    'Q8', 'RAV4', 'Range Rover', 'Range Rover Sport', 'Renegade', 'S2', 'SQ5', 'SW4',
    'Samurai', 'Sandero', 'Sandero Stepway', 'Santa Fe', 'Seltos', 'Serie 4', 'Sorento',
    'Soul', 'Spin', 'Sportage', 'Stelvio', 'T-Cross', 'Taos', 'Terios', 'Terrano II',
    'Territory', 'Tiggo', 'Tiggo 2', 'Tiggo 3', 'Tiggo 4', 'Tiggo 4 Pro', 'Tiggo 5',
    'Tiggo 8 Pro', 'Tiguan', 'Tiguan Allspace', 'Touareg', 'Tracker', 'Trailblazer',
    'Trooper', 'Tucson', 'UX', 'Vitara', 'Wrangler', 'X-Terra', 'X-Trail', 'X1', 'X2',
    'X25', 'X3', 'X35', 'X4', 'X5', 'X55', 'X6', 'X70', 'XC40', 'XC60', 'q5 sportback'
]

model_columns = pd.DataFrame(0, index=raw_dataset.index, columns=known_models)
raw_dataset = pd.concat([raw_dataset, model_columns], axis=1)

for i, modelo_input in enumerate(raw_dataset['Modelo']):
    dists = [levenshtein_distance(str(modelo_input).lower(), model.lower()) for model in known_models]
    idx_min = np.argmin(dists)
    matched_model = known_models[idx_min]
    raw_dataset.at[i, matched_model] = 1

Se hace one-hot encoding de las columnas 'Tipo de combustible', 'Transmisión', 'Tipo de vendedor' y 'Moneda'

In [74]:
dummies_combu = pd.get_dummies(raw_dataset['Tipo de combustible'], prefix='Combustible')
dummies_combu = dummies_combu.astype(int)
raw_dataset = pd.concat([raw_dataset, dummies_combu], axis=1)
raw_dataset = raw_dataset.drop(columns=['Tipo de combustible'])

dummies_tran = pd.get_dummies(raw_dataset['Transmisión'], prefix='Transmisión')
dummies_tran = dummies_tran.astype(int)
raw_dataset = pd.concat([raw_dataset, dummies_tran], axis=1)
raw_dataset = raw_dataset.drop(columns=['Transmisión'])

dummies_vend = pd.get_dummies(raw_dataset['Tipo de vendedor'], prefix='Vendedor')
dummies_vend = dummies_vend.astype(int)
raw_dataset = pd.concat([raw_dataset, dummies_vend], axis=1)
raw_dataset = raw_dataset.drop(columns=['Tipo de vendedor'])

dummies_moneda = pd.get_dummies(raw_dataset['Moneda'], prefix='Moneda')
dummies_moneda = dummies_moneda.astype(int)
raw_dataset = pd.concat([raw_dataset, dummies_moneda], axis=1)
raw_dataset = raw_dataset.drop(columns=['Moneda'])

Extraer los lítros del motor de las columnas en las que puede estar

In [75]:
import re

def extraer_litros(motor_str):
    if pd.isna(motor_str):
        return np.nan
    match = re.search(r'(\d+[.,]?\d*)', motor_str)
    if match:
        valor = float(match.group(1).replace(',', '.'))
        if 0 < valor < 7:
            return valor
    return np.nan

raw_dataset['LitrosMotor'] = raw_dataset['Motor'].apply(extraer_litros)

df = raw_dataset.copy()

df['LitrosMotor_extraido_de_Titulo'] = 0
df['LitrosMotor_extraido_de_Version'] = 0

filtro_nulos = df['LitrosMotor'].isna()
litros_desde_titulo = df.loc[filtro_nulos, 'Título'].apply(extraer_litros)
indices_titulo = litros_desde_titulo[litros_desde_titulo.notna()].index
df.loc[indices_titulo, 'LitrosMotor'] = litros_desde_titulo.loc[indices_titulo]
df.loc[indices_titulo, 'LitrosMotor_extraido_de_Titulo'] = 1

filtro_nulos = df['LitrosMotor'].isna()
litros_desde_version = df.loc[filtro_nulos, 'Versión'].apply(extraer_litros)
indices_version = litros_desde_version[litros_desde_version.notna()].index
df.loc[indices_version, 'LitrosMotor'] = litros_desde_version.loc[indices_version]
df.loc[indices_version, 'LitrosMotor_extraido_de_Version'] = 1

raw_dataset = df

Se droppean las features ya procesadas.

In [76]:
raw_dataset.drop(columns=['Marca', 'Motor', 'Color', 'Unnamed: 0'], inplace=True)

# Guardar el dataset

In [77]:
raw_dataset.to_csv('data/pf_suvs_i302_1s2025_to_colab.csv', index=False)